# Serve Docs
This is a utlity to serve the docs locally.

1. Run a CLUE instance locally
2. Configure and run the cell below.
3. A list of links to to the documents as raw json and in document viewer will be visible at this url {{ insert url here }}




In [1]:
import os
import http.server

# server class
class CustomHTTPRequestHandler(http.server.SimpleHTTPRequestHandler):
    def end_headers(self):
        self.send_header('Access-Control-Allow-Origin', '*')
        self.send_header('Access-Control-Allow-Methods', 'GET')
        self.send_header('Access-Control-Allow-Headers', 'Content-Type')
        if self.path.endswith(".json"):
            self.send_header("Content-Type", 'application/json')
        else:
            self.send_header("Content-Type", 'text/html')
        http.server.SimpleHTTPRequestHandler.end_headers(self)

    def do_GET(self):
        if self.path == '/':
            self.path = '/index.html'
        return http.server.SimpleHTTPRequestHandler.do_GET(self)

# playing around in jupyter can mess up os current working directory
# so make sure we are in the right place
starting_dir = os.getcwd()
if starting_dir.endswith('documents'):
    os.chdir('..')

current_dir = os.getcwd()
if not current_dir.endswith('docsfiles'):
    print('This script must be run from the `docsfiles` directory. Quit VSC and restart from the correct directory.')
    exit()

documents_dir = "./documents/"
os.chdir(documents_dir)

# write an html file to serve
with open('index.html', 'w') as f:
    style="<style>table {font-size: 12px; font-family: arial, sans-serif;border-collapse: collapse;}td, th {border: 1px solid #dddddd;text-align: left;padding: 4px;}tr:nth-child(even) {background-color: #dddddd;}</style>"
    html_top = "<html><body>"
    heading = "<h1>Documents:</h1>"
    table_start = "<table><thead><tr><th>Document</th><th>JSON Link</th><th>Document View</th></tr></thead><tbody>"
    html_start = style + html_top + heading + table_start
    f.write(html_start)

    for document in os.listdir('.'):
        doc_link = "http://localhost:8080/" + document + ".html"
        title_cell = f'<tr><td>{document}</td>'
        json_cell = f'<td><a target="_blank" href="{document}">json</a></td>'
        doc_cell = f'<td><a target="_blank" href="{doc_link}">doc</a></td></tr>'
        f.write(title_cell + json_cell + doc_cell)

    table_end = "</tbody></table>"
    html_bottom = "</body></html>"
    html_end = table_end + html_bottom
    f.write(html_end)

handler = CustomHTTPRequestHandler
httpd = http.server.HTTPServer(('localhost', 8083), handler)

print(f"Serving HTTP on localhost port 8083...")
print(f"open http://localhost:8083/ in your browser")

httpd.serve_forever()



Serving HTTP on localhost port 8083...
open http://localhost:8083/ in your browser


127.0.0.1 - - [14/Nov/2023 22:03:33] code 404, message File not found
127.0.0.1 - - [14/Nov/2023 22:03:33] "GET /document2.txt HTTP/1.1" 404 -
127.0.0.1 - - [14/Nov/2023 22:04:50] code 404, message File not found
127.0.0.1 - - [14/Nov/2023 22:04:50] "GET /document2.txt HTTP/1.1" 404 -
127.0.0.1 - - [14/Nov/2023 22:04:50] code 404, message File not found
127.0.0.1 - - [14/Nov/2023 22:04:50] "GET /favicon.ico HTTP/1.1" 404 -
